In [2]:
import sys
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QComboBox, 
    QPushButton, QTableWidget, QTableWidgetItem, QHBoxLayout
)
from PyQt5.QtGui import QFont
from PyQt5.QtCore import Qt, QThread, pyqtSignal
import sqlite3
import pandas as pd

# Função para inicializar o banco de dados
def inicializar_banco():
    conexao = sqlite3.connect("cambio.db")
    cursor = conexao.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS cambio (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            moeda_origem TEXT,
            moeda_destino TEXT,
            data DATE,
            taxa REAL,
            UNIQUE(moeda_origem, moeda_destino, data)
        )
    """)
    conexao.commit()
    conexao.close()

# Função para salvar dados no banco de dados
def salvar_no_banco(moeda_origem, moeda_destino, datas, taxas):
    conexao = sqlite3.connect("cambio.db")
    cursor = conexao.cursor()
    for data, taxa in zip(datas, taxas):
        try:
            cursor.execute("""
                INSERT INTO cambio (moeda_origem, moeda_destino, data, taxa)
                VALUES (?, ?, ?, ?)
            """, (moeda_origem, moeda_destino, data.strftime("%Y-%m-%d"), taxa))
        except sqlite3.IntegrityError:  # Já existe, apenas atualiza
            cursor.execute("""
                UPDATE cambio SET taxa = ?
                WHERE moeda_origem = ? AND moeda_destino = ? AND data = ?
            """, (taxa, moeda_origem, moeda_destino, data.strftime("%Y-%m-%d")))
    conexao.commit()
    conexao.close()

def carregar_dados_filtrados(moeda_origem, moeda_destino):
    conexao = sqlite3.connect("cambio.db")
    query = """
        SELECT * FROM cambio
        WHERE moeda_origem = ? AND moeda_destino = ?
    """
    try:
        # Executa a consulta filtrada e converte os resultados em DataFrame
        df = pd.read_sql_query(query, conexao, params=(moeda_origem, moeda_destino))
        return df
    except Exception as e:
        print(f"Erro ao carregar dados filtrados do banco: {e}")
        return None
    finally:
        conexao.close()

# Função para obter dados históricos desde o início do ano
def obter_dados_desde_inicio_ano(moeda_origem, moeda_destino):
    par_moeda = f"{moeda_origem}{moeda_destino}=X"
    dados = yf.Ticker(par_moeda)
    inicio_ano = datetime.datetime(datetime.datetime.now().year, 1, 1)
    historico = dados.history(start=inicio_ano, end=datetime.datetime.now())
    
    if not historico.empty:
        datas = historico.index.to_list()
        taxas = historico['Close'].to_list()
        return datas, taxas
    else:
        return None, None

def obter_taxa_real(moeda_origem, moeda_destino):
    """
    Obtém a taxa de câmbio atual entre duas moedas usando Yahoo Finance.
    """
    par_moeda = f"{moeda_origem}{moeda_destino}=X"
    dados = yf.Ticker(par_moeda)
    taxa_atual = dados.history(period="1d")  # Obtem a taxa mais recente

    if not taxa_atual.empty:
        return taxa_atual['Close'].iloc[-1]  # Último preço de fechamento
    else:
        return None

def obter_dados_historicos(moeda_origem, moeda_destino):
    """
    Obtém o histórico de taxas de câmbio entre duas moedas usando Yahoo Finance.
    """
    par_moeda = f"{moeda_origem}{moeda_destino}=X"
    dados = yf.Ticker(par_moeda)
    historico = dados.history(period="1mo")  # Último mês de dados

    if not historico.empty:
        datas = historico.index.to_list()[-30:]  # Últimos 30 registros
        taxas = historico['Close'].to_list()[-30:]
        return datas, taxas
    else:
        return None, None

from PyQt5.QtCore import QThread, pyqtSignal

# Classe para buscar a taxa de câmbio em um thread separado
class ConversorThread(QThread):
    # Sinais para enviar dados de volta para a interface principal
    taxa_obtida = pyqtSignal(float)
    erro_ocorrido = pyqtSignal(str)

    def __init__(self, moeda_origem, moeda_destino):
        super().__init__()
        self.moeda_origem = moeda_origem
        self.moeda_destino = moeda_destino

    def run(self):
        """
        Código executado em um thread separado.
        """
        try:
            # Obtém a taxa de câmbio atual
            taxa = obter_taxa_real(self.moeda_origem, self.moeda_destino)
            if taxa is not None:
                self.taxa_obtida.emit(taxa)  # Emite o sinal com a taxa obtida
            else:
                self.erro_ocorrido.emit("Erro ao obter taxa de câmbio.")
        except Exception as e:
            self.erro_ocorrido.emit(str(e))  # Emite um sinal de erro


class ConversorMoedas(QWidget):
    def __init__(self):
        super().__init__()

        # Configuração da janela
        self.setWindowTitle("Conversor de Moedas com Taxas Reais e Gráfico Histórico")
        self.setGeometry(100, 100, 800, 600)

        # Layout principal
        main_layout = QHBoxLayout()  # Layout horizontal para dividir a tela

        # Layout para o conversor e histórico
        left_layout = QVBoxLayout()

        # Título
        self.label_titulo = QLabel("Conversor de Moedas")
        self.label_titulo.setFont(QFont("Arial", 16, QFont.Bold))
        self.label_titulo.setAlignment(Qt.AlignCenter)
        left_layout.addWidget(self.label_titulo)

        # Entrada de valor
        self.label_valor = QLabel("Valor a Converter:")
        self.label_valor.setFont(QFont("Arial", 12))
        left_layout.addWidget(self.label_valor)
        self.entry_valor = QLineEdit()
        left_layout.addWidget(self.entry_valor)

        # Seleção de moeda de origem
        self.label_origem = QLabel("Moeda de Origem:")
        self.label_origem.setFont(QFont("Arial", 12))
        left_layout.addWidget(self.label_origem)
        self.combo_origem = QComboBox()
        self.combo_origem.addItems(["USD", "EUR", "BRL", "GBP", "JPY", "CAD", "AUD"])
        left_layout.addWidget(self.combo_origem)

        # Seleção de moeda de destino
        self.label_destino = QLabel("Moeda de Destino:")
        self.label_destino.setFont(QFont("Arial", 12))
        left_layout.addWidget(self.label_destino)
        self.combo_destino = QComboBox()
        self.combo_destino.addItems(["USD", "EUR", "BRL", "GBP", "JPY", "CAD", "AUD"])
        left_layout.addWidget(self.combo_destino)

        # Taxa de câmbio atual
        self.label_taxa = QLabel("Taxa de Câmbio Atual: -")
        self.label_taxa.setFont(QFont("Arial", 10))
        self.label_taxa.setStyleSheet("color: blue;")
        left_layout.addWidget(self.label_taxa)

        # Botão de conversão
        self.btn_converter = QPushButton("Converter")
        self.btn_converter.setFont(QFont("Arial", 12))
        self.btn_converter.clicked.connect(self.converter_moeda)
        left_layout.addWidget(self.btn_converter)

        # Botão para salvar série temporal no banco
        self.btn_salvar_serie = QPushButton("Salvar Série Temporal no Banco")
        self.btn_salvar_serie.clicked.connect(self.salvar_serie_temporal)
        left_layout.addWidget(self.btn_salvar_serie)

        # Resultado
        self.label_resultado = QLabel("Resultado: -")
        self.label_resultado.setFont(QFont("Arial", 12))
        left_layout.addWidget(self.label_resultado)

        # Histórico de conversões
        self.label_historico = QLabel("Histórico de Conversões")
        self.label_historico.setFont(QFont("Arial", 12, QFont.Bold))
        left_layout.addWidget(self.label_historico)

        self.table_historico = QTableWidget()
        self.table_historico.setRowCount(0)
        self.table_historico.setColumnCount(3)
        self.table_historico.setHorizontalHeaderLabels(["Data/Hora", "Valor", "Resultado"])
        left_layout.addWidget(self.table_historico)

        # Atualizar taxas
        self.btn_atualizar_taxa = QPushButton("Atualizar Taxa de Câmbio")
        self.btn_atualizar_taxa.clicked.connect(self.atualizar_taxa)
        left_layout.addWidget(self.btn_atualizar_taxa)

        # Layout para o gráfico
        self.figure = plt.figure()
        self.canvas = FigureCanvas(self.figure)
        
        right_layout = QVBoxLayout()
        right_layout.addWidget(self.canvas)
        
        main_layout.addLayout(left_layout)
        main_layout.addLayout(right_layout)

        # Define a proporção de 1:1 para cada lado
        main_layout.setStretch(0, 1)  # O layout da esquerda ocupa 50%
        main_layout.setStretch(1, 1)  # O layout da direita ocupa 50%

        self.setLayout(main_layout)
    
    def salvar_serie_temporal(self):
            moeda_origem = self.combo_origem.currentText()
            moeda_destino = self.combo_destino.currentText()
            datas, taxas = obter_dados_desde_inicio_ano(moeda_origem, moeda_destino)
    
            if datas and taxas:
                salvar_no_banco(moeda_origem, moeda_destino, datas, taxas)
                self.label_resultado.setText("Série temporal salva no banco com sucesso!")
            else:
                self.label_resultado.setText("Erro ao obter série temporal.")

    def atualizar_taxa(self):
        # Obtém as moedas selecionadas
        moeda_origem = self.combo_origem.currentText()
        moeda_destino = self.combo_destino.currentText()

        # Cria e inicia o thread para buscar a taxa de câmbio
        self.thread = ConversorThread(moeda_origem, moeda_destino)
        self.thread.taxa_obtida.connect(self.exibir_taxa)  # Conecta o sinal à função de sucesso
        self.thread.erro_ocorrido.connect(self.exibir_erro)  # Conecta o sinal à função de erro
        self.thread.start()  # Inicia o thread

        # Atualiza a interface enquanto busca
        self.label_taxa.setText("Obtendo taxa de câmbio...")

    def exibir_taxa(self, taxa):
        moeda_origem = self.combo_origem.currentText()
        moeda_destino = self.combo_destino.currentText()
        self.label_taxa.setText(f"Taxa de Câmbio Atual: 1 {moeda_origem} = {taxa:.4f} {moeda_destino}")

    def exibir_erro(self, mensagem):
        self.label_taxa.setText(mensagem)


    def converter_moeda(self):
        try:
            # Obtendo valores de entrada
            valor = float(self.entry_valor.text())
            moeda_origem = self.combo_origem.currentText()
            moeda_destino = self.combo_destino.currentText()

            # Obtenção da taxa de câmbio
            taxa = obter_taxa_real(moeda_origem, moeda_destino)
            if taxa:
                resultado = valor * taxa
                self.label_resultado.setText(f"Resultado: {resultado:.2f} {moeda_destino}")
                self.label_taxa.setText(f"Taxa de Câmbio Atual: 1 {moeda_origem} = {taxa:.4f} {moeda_destino}")
                
                # Adicionando ao histórico
                self.adicionar_historico(valor, moeda_origem, resultado, moeda_destino)
                
                # Exibe o gráfico automaticamente
                self.plot_historico()
            else:
                self.label_resultado.setText("Erro ao obter taxa de câmbio.")
        except ValueError:
            self.label_resultado.setText("Insira um valor válido.")


    def adicionar_historico(self, valor, moeda_origem, resultado, moeda_destino):
        row_position = self.table_historico.rowCount()
        self.table_historico.insertRow(row_position)

        # Data/Hora
        data_hora = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.table_historico.setItem(row_position, 0, QTableWidgetItem(data_hora))

        # Valor convertido
        valor_texto = f"{valor} {moeda_origem}"
        self.table_historico.setItem(row_position, 1, QTableWidgetItem(valor_texto))

        # Resultado
        resultado_texto = f"{resultado:.2f} {moeda_destino}"
        self.table_historico.setItem(row_position, 2, QTableWidgetItem(resultado_texto))

    def plot_historico(self):
        moeda_origem = self.combo_origem.currentText()
        moeda_destino = self.combo_destino.currentText()
        datas, taxas = obter_dados_historicos(moeda_origem, moeda_destino)

        if datas and taxas:
            # Limpa a figura e configura o gráfico
            self.figure.clear()
            ax = self.figure.add_subplot(111)
            ax.plot(datas, taxas, marker="o", linestyle="-")
            ax.set_title(f"Histórico de Taxa de Câmbio: {moeda_origem} para {moeda_destino} - Últimos 30 dias")
            ax.set_xlabel("Data")
            ax.set_ylabel(f"Taxa de Câmbio ({moeda_origem}/{moeda_destino})")
            ax.grid(True)
            
            # Formatação da data para mês e dia
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))
            self.canvas.draw()
        else:
            self.label_resultado.setText("Erro ao obter dados históricos.")

# Inicialização da aplicação
app = QApplication(sys.argv)
window = ConversorMoedas()
window.show()
sys.exit(app.exec_())



SystemExit: 0

c:\Users\lucas\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<Figure size 383x574 with 0 Axes>

In [5]:
def consultabanco(moeda_origem, moeda_destino):
    if __name__ == "__main__":
        print("Carregando dados filtrados do banco...")

        # Especifica as moedas de origem e destino
        moeda1 = moeda_origem
        moeda2 = moeda_destino

        # Carrega os dados filtrados
        df_filtrado = carregar_dados_filtrados(moeda_origem, moeda_destino)

        # Verifica o resultado
        if df_filtrado is not None and not df_filtrado.empty:
            print(f"Dados filtrados para {moeda_origem} -> {moeda_destino}:")
            print(df_filtrado)
        else:
            print(f"Nenhum dado encontrado para {moeda_origem} -> {moeda_destino}.")

consultabanco('USD','EUR')



Carregando dados filtrados do banco...
Dados filtrados para USD -> EUR:
      id moeda_origem moeda_destino        data     taxa
0      1          USD           EUR  2024-01-01  0.90450
1      2          USD           EUR  2024-01-02  0.90590
2      3          USD           EUR  2024-01-03  0.91393
3      4          USD           EUR  2024-01-04  0.91510
4      5          USD           EUR  2024-01-05  0.91346
..   ...          ...           ...         ...      ...
229  230          USD           EUR  2024-11-15  0.94966
230  231          USD           EUR  2024-11-18  0.94933
231  232          USD           EUR  2024-11-19  0.94385
232  233          USD           EUR  2024-11-20  0.94272
233  234          USD           EUR  2024-11-21  0.94821

[234 rows x 5 columns]
